In [1]:
import os
import hardposit as hp
import fixed2float as fi
from pathlib import Path
import re

In [2]:
!date

Mon  8 May 2023 08:55:51 CEST


In [3]:
import fixed2float
fixed2float.__version__

'4.0.0-rc1'

In [4]:
N, ES = 16, 1
M, B = 31, 64

In [5]:
with open(Path("../tb_core_op_fma.log"), 'r') as f:
  log = f.read()
print(log)

0x3524
(0x0772, 0x5e81) 0x00000000b6430e40
(0xa0b2, 0xd609) 0x000000024cac2f40
(0x6e10, 0x5663) 0x00000029a6b22f40
(0xf530, 0x7b0d) 0x00000024748e2f40
(0xe7b5, 0x998d) 0x0000002564359140
(0x6db6, 0x8465) 0xfffff3c997b59140
(0xcb14, 0x5212) 0xfffff3c81da4f940
(0x9cdf, 0xe301) 0xfffff3c93ee8b740
(0xaebb, 0xcd0d) 0xfffff3ca86273640
(0x3beb, 0xf176) 0xfffff3ca7abe4b60
(0x4441, 0xcd3d) 0xfffff3c9bcbe89e0
(0x4040, 0x57ed) 0xfffff3ccc65509e0
(0xf18a, 0xf78c) 0xfffff3ccc68e94d0
(0xe303, 0xe9f9) 0xfffff3cccc401c28
(0x55fc, 0x24c6) 0xfffff3cdb0989028
(0x945d, 0x84c5) 0xfffffd3394d89028
(0xc21d, 0xd2aa) 0xfffffd340270b868
(0x2a6a, 0xf7e5) 0xfffffd3400bef3b0
(0xb116, 0x7277) 0xfffffd020a0d73b0
(0x353f, 0xd612) 0xfffffd01c5308170



In [6]:
logs = log.splitlines()
print(logs)

['0x3524', '(0x0772, 0x5e81) 0x00000000b6430e40', '(0xa0b2, 0xd609) 0x000000024cac2f40', '(0x6e10, 0x5663) 0x00000029a6b22f40', '(0xf530, 0x7b0d) 0x00000024748e2f40', '(0xe7b5, 0x998d) 0x0000002564359140', '(0x6db6, 0x8465) 0xfffff3c997b59140', '(0xcb14, 0x5212) 0xfffff3c81da4f940', '(0x9cdf, 0xe301) 0xfffff3c93ee8b740', '(0xaebb, 0xcd0d) 0xfffff3ca86273640', '(0x3beb, 0xf176) 0xfffff3ca7abe4b60', '(0x4441, 0xcd3d) 0xfffff3c9bcbe89e0', '(0x4040, 0x57ed) 0xfffff3ccc65509e0', '(0xf18a, 0xf78c) 0xfffff3ccc68e94d0', '(0xe303, 0xe9f9) 0xfffff3cccc401c28', '(0x55fc, 0x24c6) 0xfffff3cdb0989028', '(0x945d, 0x84c5) 0xfffffd3394d89028', '(0xc21d, 0xd2aa) 0xfffffd340270b868', '(0x2a6a, 0xf7e5) 0xfffffd3400bef3b0', '(0xb116, 0x7277) 0xfffffd020a0d73b0', '(0x353f, 0xd612) 0xfffffd01c5308170']


In [7]:
pattern = r"0x[0-9a-z]+"

acc = 0
for item in logs:
  results = re.findall(pattern, item)
  if len(results) == 1:
    p3 = hp.from_bits(int(results[0], 16), N, ES)
    acc += p3.eval()
  else:
    p1 = hp.from_bits(int(results[0], 16), N, ES)
    p2 = hp.from_bits(int(results[1], 16), N, ES)
    print(p1.eval(), p2.eval(), (p1*p2).eval())

    fixed = fi.from_bits(int(results[2], 16), M, B)
    print(f"fixed={fixed}")
    acc += p1.eval() * p2.eval()
    print(f"acc={acc}")
    print()

    assert acc == fixed.eval()

0.013458251953125 3.81298828125 0.05133056640625
fixed=0000000000000000000000000000000010110110010000110000111001000000 Fx<31,64>(3057847872) 0.7119606882333755
acc=0.7119606882333755

-3.9130859375 -0.40570068359375 1.587646484375
fixed=0000000000000000000000000000001001001100101011000010111101000000 Fx<31,64>(9876287296) 2.2995023280382156
acc=2.2995023280382156

14.0625 2.79833984375 39.34375
fixed=0000000000000000000000000010100110100110101100100010111101000000 Fx<31,64>(178890354496) 41.65115638077259
acc=41.65115638077259

-0.026611328125 195.25 -5.1953125
fixed=0000000000000000000000000010010001110100100011100010111101000000 Fx<31,64>(156574297920) 36.45529456436634
acc=36.45529456436634

-0.12957763671875 -7.224609375 0.9361572265625
fixed=0000000000000000000000000010010101100100001101011001000101000000 Fx<31,64>(160595022144) 37.391442373394966
acc=37.391442373394966

13.7109375 -230.75 -3168.0
fixed=1111111111111111111100111100100110010111101101011001000101000000 Fx<31,64>(18